In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy import misc
import imageio
import os
import pandas as pd
import mahotas as mh

In [3]:
# folder path
dir_path = r'BestSegmentations/'
# list to store files
res = [] # files name
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)

In [3]:
glam_iou_otsu = pd.DataFrame(columns=['image_index', 'threshold_otsu', 'IoU'], index=range(370))
for i in range(0, 370, 1):
  segm1 = imageio.imread(('BestSegmentations/' + res[i]))
  res1 = str(res[i])
  glam_otsu = cv2.imread(('GrayHeatmapsGlam/' + res1.split(".")[0] + '_CAM2.png'), cv2.IMREAD_UNCHANGED)
  assert np.shape(segm1) == np.shape(glam_otsu)
  otsu_threshold, image_otsu = cv2.threshold(glam_otsu, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 
  intersection = np.logical_and(image_otsu, segm1)
  union = np.logical_or(image_otsu, segm1)
  iou = np.sum(intersection) / np.sum(union)
  glam_iou_otsu.loc[i] = (res1.split(".")[0], otsu_threshold, iou)
  # Save image with contours--------------------------------------------------------------------------------------------
  #orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'),0)
  #contours_seg, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  #contours_otsu, hierarchy1 = cv2.findContours(image_otsu, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  #cv2.drawContours(orig, contours_seg, -1, (0,255,0), 6)
  #cv2.drawContours(orig, contours_otsu, -1, (255,255,0), 6)
  #cv2.imwrite(('ImageOtsu/Glam/' + res1.split(".")[0] + '_CAM2OT.png'), orig)

In [4]:
glam_iou_otsu

,image_index,threshold_otsu,IoU
0,123ca_187_L_CC,111,0.0434279
1,173ca_256_R_MLO,112,0.0493894
2,168ca_248_L_MLO,0,0
3,137ca_208_R_CC,105,0
4,133ca_203_L_MLO,105,0
...,...,...,...
365,14ca_19_R_MLO,111,0.137742
366,140ca_211_L_MLO,116,0.0358601
367,23ca_38_L_CC,107,0.124066
368,13ca_18_R_CC,108,0


In [5]:
np.sum(glam_iou_otsu['IoU'])/370
# Antes 0.07516612911411437

0.08178740807126911

In [7]:
glam_iou = pd.DataFrame(columns=['image_index', 'threshold', 'IoU'])
for i in range(0, 370, 1):
  segm1 = imageio.imread(('BestSegmentations/' + res[i]))
  res1 = str(res[i])
  glam = cv2.imread(('GrayHeatmapsGlam/' + res1.split(".")[0] + '_CAM2.png'), cv2.IMREAD_UNCHANGED)
  assert np.shape(segm1) == np.shape(glam)
  iou_score = []
  for n in range(0, 255, 1):
    th , image = cv2.threshold(glam, n, 255, cv2.THRESH_BINARY) 
    intersection = np.logical_and(image, segm1)
    union = np.logical_or(image, segm1)
    iou = np.sum(intersection) / np.sum(union)
    iou_score.append(iou)
  threshold = iou_score.index(np.max(iou_score))
  IoU = np.max(iou_score)
  glam_iou.loc[i] = (res1.split(".")[0], threshold, IoU)
  # Save image with contours--------------------------------------------------------------------------------------------
  #orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'),0)
  #th , image = cv2.threshold(glam, threshold, 255, cv2.THRESH_BINARY)
  #contours_seg, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  #contours_sweep, hierarchy1 = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  #cv2.drawContours(orig, contours_seg, -1, (0,255,0), 6)
  #cv2.drawContours(orig, contours_sweep, -1, (255,255,0), 6)
  #cv2.imwrite(('ImageSweep/Glam/' + res1.split(".")[0] + '_CAM2SW.png'), orig)


In [ ]:
glam_iou 

In [ ]:
np.sum(glam_iou['IoU'])/370
# antes 0.12014099804687912

0.12856702572279918

In [4]:
glam_iou_otsu_ = pd.DataFrame(columns=['image_index', 'threshold_otsu', 'IoU'])
glam_iou_no_ = pd.DataFrame(columns=['image_index', 'threshold_otsu'])
otsu_ = 0
no_ = 0
for i in range(0, 370, 1):
    segm1 = imageio.imread(('BestSegmentations/' + res[i]))
    res1 = str(res[i])
    orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'))
    glam_otsu = cv2.imread(('GrayHeatmapsGlam/' + res1.split(".")[0] + '_CAM2.png'), cv2.IMREAD_UNCHANGED)
    assert np.shape(segm1) == np.shape(glam_otsu)
    otsu_threshold, image_otsu = cv2.threshold(glam_otsu, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    inter_out = np.sum(np.logical_and(image_otsu, segm1))
    if inter_out != 0: 
        labelled, nr_objects = mh.label(image_otsu)
        new_image = np.zeros(np.shape(image_otsu))
        for i in range(1, nr_objects + 1, 1):
            mask = (labelled == i)
            inter = np.sum(np.logical_and(mask, segm1))
            if inter != 0:
                new_image = np.logical_or(new_image, mask)
        new_image = np.uint8(255*new_image)
        new_image = imageio.core.util.Array(new_image)
        intersection = np.logical_and(new_image, segm1)
        union = np.logical_or(new_image, segm1)
        iou = np.sum(intersection) / np.sum(union)
        glam_iou_otsu_.loc[otsu_] = (res1.split(".")[0], otsu_threshold, iou)
        otsu_ += 1
        contours, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours1, hierarchy1 = cv2.findContours(new_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(orig, contours, -1, (255,255,0), 10)
        cv2.drawContours(orig, contours1, -1, (0,255,0), 10)
        cv2.imwrite(('ImageOtsu/Glam/' + res1.split(".")[0] + '_CAM2OT.png'), orig)
    else:
        glam_iou_no_.loc[no_] = (res1.split(".")[0], otsu_threshold)
        no_ += 1

In [5]:
glam_iou_otsu_

,image_index,threshold_otsu,IoU
0,123ca_187_L_CC,111.0,0.043444
1,173ca_256_R_MLO,112.0,0.055074
2,151ca_225_L_MLO,105.0,0.152928
3,28ca_45_R_CC,111.0,0.587246
4,65ca_99_L_MLO,107.0,0.448675
...,...,...,...
141,154ca_228_L_CC,104.0,0.069973
142,14ca_19_R_MLO,111.0,0.137742
143,140ca_211_L_MLO,116.0,0.035867
144,23ca_38_L_CC,107.0,0.134805


In [6]:
print('IoU Glam Otsu = ', np.sum(glam_iou_otsu_['IoU'])/otsu_)

IoU Glam Otsu =  0.228353808203027


In [8]:
glam_iou_sw_ = pd.DataFrame(columns=['image_index', 'threshold', 'IoU'])
glam_iou_n_ = pd.DataFrame(columns=['image_index', 'threshold_otsu'])
sw_ = 0
n_ = 0

for i in range(0, 370, 1):
    segm1 = imageio.imread(('BestSegmentations/' + res[i]))
    res1 = str(res[i])
    orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'))
    glam_sw = cv2.imread(('GrayHeatmapsGlam/' + res1.split(".")[0] + '_CAM2.png'), cv2.IMREAD_UNCHANGED)
    assert np.shape(segm1) == np.shape(glam_sw)
    iou_score = []
    for n in range(0, 255, 1):
        th , image = cv2.threshold(glam_sw, n, 255, cv2.THRESH_BINARY)      
        intersection = np.logical_and(image, segm1)
        union = np.logical_or(image, segm1)
        iou = np.sum(intersection) / np.sum(union)
        iou_score.append(iou)
    threshold = iou_score.index(np.max(iou_score))
    IoU = np.max(iou_score)
    th , image_sw = cv2.threshold(glam_sw, threshold, 255, cv2.THRESH_BINARY) 
    inter_out = np.sum(np.logical_and(image_sw, segm1))
    if inter_out != 0: 
        labelled, nr_objects = mh.label(image_sw)
        new_image = np.zeros(np.shape(image_sw))
        for i in range(1, nr_objects + 1, 1):
            mask = (labelled == i)
            inter = np.sum(np.logical_and(mask, segm1))
            if inter != 0:
                new_image = np.logical_or(new_image, mask)
        new_image = np.uint8(255*new_image)
        new_image = imageio.core.util.Array(new_image)
        intersection = np.logical_and(new_image, segm1)
        union = np.logical_or(new_image, segm1)
        iou = np.sum(intersection) / np.sum(union)
        glam_iou_sw_.loc[sw_] = (res1.split(".")[0], th, iou)
        sw_ += 1
        contours, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours1, hierarchy1 = cv2.findContours(new_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(orig, contours, -1, (255,255,0), 10)
        cv2.drawContours(orig, contours1, -1, (0,255,0), 10)
        cv2.imwrite(('ImageSweep/Glam/' + res1.split(".")[0] + '_CAM2SW.png'), orig)
    else:
        glam_iou_n_.loc[n_] = (res1.split(".")[0], otsu_threshold)
        n_ += 1

In [9]:
glam_iou_sw_

,image_index,threshold,IoU
0,123ca_187_L_CC,0.0,0.197025
1,173ca_256_R_MLO,12.0,0.092306
2,151ca_225_L_MLO,3.0,0.378528
3,28ca_45_R_CC,180.0,0.615733
4,162ca_241_L_MLO,0.0,0.028109
...,...,...,...
143,154ca_228_L_CC,0.0,0.387507
144,14ca_19_R_MLO,0.0,0.426144
145,140ca_211_L_MLO,0.0,0.188573
146,23ca_38_L_CC,0.0,0.253322


In [10]:
print('IoU Glam Sweep = ', np.sum(glam_iou_sw_['IoU'])/sw_)

IoU Glam Sweep =  0.35887057294718105
